In [ ]:
import time as time
import pandas as pd
import os

from selenium import webdriver 

In [ ]:
url = 'https://search.cpsa.ca/PhysicianSearch'

In [ ]:
# Helper function that will help us write data to disk by appending
def appendToFile(filePath, df):
    header = True    
    # If file exist don't append the header 
    if os.path.isfile(filePath):
        header = False
        
    df.to_csv(filePath, mode='a', header=header, index=False)

In [ ]:
option = webdriver.ChromeOptions()
option.add_argument('-incognito')

browser = webdriver.Chrome(executable_path='../chromedriver', chrome_options=option)

In [ ]:
# Tell browser to go to search page
browser.get(url)

# Wait for the button to load
time.sleep(1)

# Click on the button for a global search
browser.find_element_by_id('MainContent_physicianSearchView_btnSearch').click()

# Takes the site on average 7s to load completely
time.sleep(7)

# Tell the browser to get the printer version so all the doctors are on page without any CSS
browser.get('https://search.cpsa.ca/PrinterFriendlyResults.aspx')

# Takes 20s for the browser to load the entire print version
time.sleep(20)

# Find the table element
table = browser.find_element_by_id('printerFriendlySearchResultsView_gvResults')

# Create the list of table rows
trList = table.find_elements_by_tag_name('tr')

# Remove the header row
trList = trList[1:]

In [ ]:
# For each row start scrapping the info
for i in range(len(trList)):

    # Checks if df exists in local scope and creates one if not
    if 'df' not in locals():
        df = pd.DataFrame()
        
    # grab the column string values at the i'th row
    status = trList[i].find_elements_by_tag_name('td')[0].text
    name = trList[i].find_elements_by_tag_name('td')[1].text
    location = trList[i].find_elements_by_tag_name('td')[2].text
    practiceDiscipline = trList[i].find_elements_by_tag_name('td')[3].text
    gender = trList[i].find_elements_by_tag_name('td')[4].text
    language = trList[i].find_elements_by_tag_name('td')[5].text
    
    # Package the variables into a dict
    data = {'Status' : status, 
            'Name' : name, 
            'Location' : location, 
            'Specialty' : practiceDiscipline, 
            'Gender' : gender, 
            'Languages' : language}
    
    # Append the data to the df
    df = df.append(data, ignore_index=True)
    
    # if 1000 items have been iterated then append the df in memory and delete df to free up ram space
    if i % 1000 == 0:
        appendToFile('./csv/AB Physician.csv', df)
        del df